# FULL IMPLEMENTATION USING PYTHON:

In [12]:
# importing libraries
import cv2
import numpy as np
from PIL import Image
import random
from matplotlib import pyplot as plt
from skimage.util import random_noise
from IPython.display import Image as imvis
from IPython.display import Video
import os
from os.path import isfile, join

# 1. Load Video and Add Noise

In [13]:
Video("demo.mp4")

In [14]:
#Function for adding Gaussian and Impulse noise
def noisy(noise_typ,img):
    if noise_typ == "gauss":
         # Generate Gaussian noise
        gauss = np.random.normal(0,1,img.size)
        gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
        # Add the Gaussian noise to the image
        img_gauss = cv2.add(img,gauss)
        return img_gauss
    elif noise_typ == "s&p":
        # Add salt-and-pepper noise to the image.
        noise_img = random_noise(img, mode='s&p',amount=0.2)

        # The above function returns a floating-point image
        # on the range [0, 1], thus we changed it to 'uint8'
        # and from [0,255]
        noise_img = np.array(255*noise_img, dtype = 'uint8')
        return noise_img

In [15]:
# Read the video from specified path
cam = cv2.VideoCapture('demo.mp4')
  
# frame
currentframe = 0

print("Adding noise....")
while(True):
      
    # reading from frame
    ret,frame = cam.read()

  
    if ret:
        # if video is still left continue creating images
        name = r"C:\Users\sree\Desktop\Fastrack\Image Processing\DA-1\frames\frame" + str(currentframe) + '.jpg'
        #print ('Creating frame number ' + str(currentframe))
  
        # writing the extracted images
        cv2.imwrite(name, frame)
  
        
        #Adding Noise

        #Add Gaussian noise
        img1 = noisy("gauss",frame)
        #Add SNP Noise
        img2 = noisy("s&p",img1)

        #save file
        name = r"C:\Users\sree\Desktop\Fastrack\Image Processing\DA-1\Noise\noise\frame" + str(currentframe) + '.jpg'

        # writing the extracted images
        cv2.imwrite(name, img2)
        
        #increasing counter so that it will
        # show how many frames are created
        currentframe += 1
          
    else:
        break
# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()

print("Noise added")

Adding noise....
Noise added


In [45]:
#Video file
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [17]:
pathIn = r"C:\Users\sree\Desktop\Fastrack\Image Processing\DA-1\Noise\noise\\"
pathOut = "noisyvideo.mp4"
fps = 25.0
print("Processing video file...")
convert_frames_to_video(pathIn,pathOut,fps)

Processing video file...


In [18]:
Video("noisyvideo.mp4")

# 2. Load Video affected with Noise and Apply de-noising Techniques

In [19]:
#MEDIAN FILTER FOR IMPULSE NOISE
#GAUSSIAN BLUR FOR GAUSSIAN NOISE

# Function to implement Median filter

In [24]:
#median blur
def median_filter(data, filter_size):
    temp = []
    indexer = filter_size // 2
    data_final = []
    data_final = np.zeros((len(data),len(data[0])))
    for i in range(len(data)):

        for j in range(len(data[0])):

            for z in range(filter_size):
                if i + z - indexer < 0 or i + z - indexer > len(data) - 1:
                    for c in range(filter_size):
                        temp.append(0)
                else:
                    if j + z - indexer < 0 or j + indexer > len(data[0]) - 1:
                        temp.append(0)
                    else:
                        for k in range(filter_size):
                            temp.append(data[i + z - indexer][j + k - indexer])

            temp.sort()
            data_final[i][j] = temp[len(temp) // 2]
            temp = []
    return data_final

In [25]:
#convolution operation
def convolution(frame, kernel):
    image_h = frame.shape[0]
    image_w = frame.shape[1]
    kernel_h = kernel.shape[0]
    kernel_w = kernel.shape[1]

    if(len(frame.shape) == 3):
        image_pad = np.pad(frame, pad_width=(
            (kernel_h // 2, kernel_h // 2),(kernel_w // 2, kernel_w // 2),(0,0)), mode='constant', constant_values=0).astype(np.float32)
    elif(len(frame.shape) == 2):
        image_pad = np.pad(frame, pad_width=(
            (kernel_h // 2, kernel_h // 2),(kernel_w // 2, kernel_w // 2)), mode='constant', constant_values=0).astype(np.float32)


    h = kernel_h // 2
    w = kernel_w // 2

    image_conv = np.zeros(image_pad.shape)

    for i in range(h, image_pad.shape[0]-h):
        for j in range(w, image_pad.shape[1]-w):
            #sum = 0
            x = image_pad[i-h:i-h+kernel_h, j-w:j-w+kernel_w]
            x = x.flatten()*kernel.flatten()
            image_conv[i][j] = x.sum()
    h_end = -h
    w_end = -w

    if(h == 0):
        return image_conv[h:,w:w_end]
    if(w == 0):
        return image_conv[h:h_end,w:]

    return image_conv[h:h_end,w:w_end]

# Function to implement Gaussian Blur

In [33]:
#Gaussian Noise
def GaussianBlurImage(image, sigma):
    #image = imread(image)
    #image = Image.open(image)
    image = np.asarray(image)
    #print(image)
    filter_size = 2 * int(4 * sigma + 0.5) + 1
    gaussian_filter = np.zeros((filter_size, filter_size), np.float32)
    m = filter_size//2
    n = filter_size//2
    
    for x in range(-m, m+1):
        for y in range(-n, n+1):
            x1 = 2*np.pi*(sigma**2)
            x2 = np.exp(-(x**2 + y**2)/(2* sigma**2))
            gaussian_filter[x+m, y+n] = (1/x1)*x2
    
    im_filtered = np.zeros_like(image, dtype=np.float32)
    for c in range(3):
        im_filtered[:, :, c] = convolution(image[:, :, c], gaussian_filter)
    return (im_filtered.astype(np.uint8))

In [34]:
# Read the video from specified path
cam = cv2.VideoCapture("noisyvideo.mp4")
  
# frame
currentframe = 0

print("Denoising in progress...")

while(True):
      
    # reading from frame
    ret,frame = cam.read()
    #print(type(frame))
  
    if ret:
        #arr = np.array(frame)
        removed_noise = cv2.medianBlur(frame, 5) #filter size 5 
        blur = GaussianBlurImage(removed_noise, 3) #filter size 3 
        
        
        #write file
        name = r"C:\Users\sree\Desktop\Fastrack\Image Processing\DA-1\smoothing\frame" + str(currentframe) + '.jpg'

        # writing the extracted images
        cv2.imwrite(name, blur)

        currentframe+=1
        
    else:
        break
# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()

print("Completed")

Denoising in progress...
Completed


In [50]:
pathIn = r"C:\Users\sree\Desktop\Fastrack\Image Processing\DA-1\smoothing\\"
pathOut = "result.mp4"
fps = 20.0
print("Processing video file...")
convert_frames_to_video(pathIn,pathOut,fps)

Processing video file...


In [51]:
Video("result.mp4")